# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

In [3]:
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///etl_db2.db')
df = pd.read_sql("SELECT * FROM etl_df2", engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    stop_words = stopwords.words("english")
    lemmatizer= WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf' , MultiOutputClassifier(AdaBoostClassifier()))])


In [7]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [7]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)



In [105]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]
    }

# cv = GridSearchCV(pipeline, param_grid=parameters)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
model_fit = pipeline.fit(X_train,y_train)
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [17]:
print(classification_report(y_test.values,y_test_pred))

             precision    recall  f1-score   support

          0       0.77      0.97      0.86      4961
          1       0.46      0.11      0.18      1131
          2       0.00      0.00      0.00        27
          3       0.49      0.17      0.25      2718
          4       0.00      0.00      0.00       523
          5       0.17      0.00      0.01       329
          6       0.00      0.00      0.00       189
          7       0.00      0.00      0.00       103
          8       0.00      0.00      0.00       232
          9       0.00      0.00      0.00         0
         10       0.10      0.00      0.00       430
         11       0.23      0.01      0.01       787
         12       0.00      0.00      0.00       583
         13       0.25      0.03      0.05       104
         14       0.25      0.01      0.01       153
         15       0.00      0.00      0.00        77
         16       0.00      0.00      0.00       231
         17       0.12      0.00      0.01   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [19]:
parameters = { 
    'clf__estimator__n_estimators': [25, 50,75],
    'clf__estimator__learning_rate' : [0.5,1,1.5,0.2]
}

In [20]:
cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1)

In [114]:
y.related.unique()

array([ 1.,  0.])

In [21]:
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [25, 50, 75], 'clf__estimator__learning_rate': [0.5, 1, 1.5, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
cv.best_params_
# {'clf__estimator__learning_rate': 0.2, 'clf__estimator__n_estimators': 25}

{'clf__estimator__learning_rate': 0.2, 'clf__estimator__n_estimators': 25}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
pipeline_improved = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf' , MultiOutputClassifier(AdaBoostClassifier(learning_rate= 0.2,n_estimators= 25)))])


In [24]:
model_fit_imp = pipeline_improved.fit(X_train,y_train)
y_train_pred_imp = pipeline_improved.predict(X_train)
y_test_pred_imp = pipeline_improved.predict(X_test)

In [25]:
print(classification_report(y_test.values,y_test_pred_imp))

             precision    recall  f1-score   support

          0       0.76      1.00      0.87      4961
          1       0.33      0.01      0.01      1131
          2       0.00      0.00      0.00        27
          3       0.50      0.00      0.00      2718
          4       0.00      0.00      0.00       523
          5       0.00      0.00      0.00       329
          6       0.00      0.00      0.00       189
          7       0.00      0.00      0.00       103
          8       0.00      0.00      0.00       232
          9       0.00      0.00      0.00         0
         10       0.00      0.00      0.00       430
         11       0.00      0.00      0.00       787
         12       0.00      0.00      0.00       583
         13       0.00      0.00      0.00       104
         14       0.00      0.00      0.00       153
         15       0.00      0.00      0.00        77
         16       0.00      0.00      0.00       231
         17       0.00      0.00      0.00   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
pipeline_new = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))#,('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3)))
    ])

In [33]:
model_fit_new = pipeline_new.fit(X_train,y_train)
y_train_pred_new = pipeline_new.predict(X_train)
y_test_pred_new = pipeline_new.predict(X_test)

In [1]:
print(classification_report(y_test.values,y_test_pred_new))

NameError: name 'classification_report' is not defined

### 9. Export your model as a pickle file

In [35]:
import pickle

In [36]:
with open ('knn_model.pkl','wb') as file:
    pickle.dump(pipeline_new,file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.